<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Data Processing for RAG with Data Prep Kit (RAY)</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2 and 3 in RAG pipeline.

Here are the processing steps:

- **pdf2parquet** : Extract text from PDF and convert them into parquet files
- **Chunk documents**: Split the PDFs into 'meaningful sections' (paragraphs, sentences ..etc)
- **Doc_ID generation**: Each chunk is assigned a uniq id, based on content and hash
- **Exact Dedup**: Chunks with exact same content are filtered out
- **Fuzzy Dedup**: Eliminate chunks that are 'very similar' content
- **Doc quality**: Scores the documents based on criteria like number of words, if it contains bad words ..etc
- **Text encoder**: Convert chunks into vectors using embedding models

## Step-1: Configuration

In [1]:
import os
from my_config import MY_CONFIG

## RAY CONFIGURATION
num_cpus_available =  os.cpu_count()
# print (num_cpus_available)
# MY_CONFIG.RAY_NUM_CPUS = num_cpus_available // 2  ## use half the available cores for processing
MY_CONFIG.RAY_NUM_CPUS =  1
# print (MY_CONFIG.RAY_NUM_CPUS)
MY_CONFIG.RAY_MEMORY_GB = 2  # GB
# MY_CONFIG.RAY_RUNTIME_WORKERS = num_cpus_available // 3
MY_CONFIG.RAY_RUNTIME_WORKERS = 2

## Step-2:  Data

We will use white papers  about LLMs.  

- [Granite Code Models](https://arxiv.org/abs/2405.04324)
- [Attention is all you need](https://arxiv.org/abs/1706.03762)

You can of course substite your own data below

### 2.1 - Download data

In [2]:
import os, sys
import shutil
from utils import download_file

## Download the data files
shutil.os.makedirs(MY_CONFIG.INPUT_DATA_DIR, exist_ok=True)

download_file (url = 'https://arxiv.org/pdf/1706.03762', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'attension.pdf' ))

download_file (url = 'https://arxiv.org/pdf/2405.04324', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'granite.pdf' ))



input/attension.pdf (2.22 MB) downloaded successfully.

input/granite.pdf (1.27 MB) downloaded successfully.


### 2.2  - Set input/output path variables for the pipeline

In [3]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

output_parquet_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '01_parquet_out')
output_chunk_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '02_chunk_out')
output_docid_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '03_docid_out')
output_exact_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '04_exact_dedupe_out')
output_fuzzy_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '05_fuzzy_dedupe_out')
output_embeddings_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '06_embeddings_out')


## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


## Step-3: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### 3.1 - Set Input/output Folder

In [4]:
STAGE = 1 

input_folder = MY_CONFIG.INPUT_DATA_DIR
output_folder =  output_parquet_dir

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-1: Processing input='input' --> output='output/01_parquet_out'


### 3.2 -  Execute 

In [5]:
%%time 

import ast
import os
import sys

from data_processing_ray.runtime.ray import RayTransformLauncher
from data_processing.utils import GB, ParamsUtils

from pdf2parquet_transform import (
    pdf2parquet_contents_type_cli_param,
    pdf2parquet_contents_types,
)
from pdf2parquet_transform_python import Pdf2ParquetPythonTransformConfiguration
from pdf2parquet_transform_ray import Pdf2ParquetRayTransformConfiguration

# create parameters
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS, "memory": MY_CONFIG.RAY_MEMORY_GB * GB}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
ingest_config = {
    pdf2parquet_contents_type_cli_param: pdf2parquet_contents_types.JSON,
}

params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    "data_files_to_use": ast.literal_eval("['.pdf']"),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    "runtime_pipeline_id": "pipeline_id",
    "runtime_job_id": "job_id",
    "runtime_code_location": ParamsUtils.convert_to_ast(code_location),
}


sys.argv = ParamsUtils.dict_to_req(d=(params | ingest_config))
# create launcher
launcher = RayTransformLauncher(Pdf2ParquetRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")


15:44:08 INFO - pdf2parquet parameters are : {'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.JSON: 'application/json'>, 'do_table_structure': True, 'do_ocr': True, 'double_precision': 8}
15:44:08 INFO - pipeline id pipeline_id
15:44:08 INFO - code location {'github': 'github', 'commit_hash': '12345', 'path': 'path'}
15:44:08 INFO - number of workers 2 worker options {'num_cpus': 1, 'memory': 2147483648, 'max_restarts': -1}
15:44:08 INFO - actor creation delay 0
15:44:08 INFO - job details {'job category': 'preprocessing', 'job name': 'pdf2parquet', 'job type': 'ray', 'job id': 'job_id'}
15:44:08 INFO - data factory data_ is using local data access: input_folder - input output_folder - output/01_parquet_out
15:44:08 INFO - data factory data_ max_files -1, n_sample -1
15:44:08 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
15:44:08 INFO - Running locally
202

✅ Stage:1 completed successfully
CPU times: user 2.18 s, sys: 1.02 s, total: 3.21 s
Wall time: 2min 56s


### 3.3 - Inspect Generated output

Here we should see one entry per input file processed

In [6]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Output dimensions (rows x columns)=  (2, 12)


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,granite.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",28,17,348,b5367177-e1c1-4643-9281-942f41bc5c87,pdf,4b4ee627dd6a21c1eb56205875a3970c3ad975864f3dcf...,655037,2024-10-08T15:46:23.368528,81.505457,granite.pdf
1,attension.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",15,4,193,89c788c3-8f54-4809-8177-d90721f28587,pdf,8b3ff3ebfc838c36fa10e6bbf90c1a5d2781f2d2583883...,135809,2024-10-08T15:45:38.327708,36.473885,attension.pdf


##  Step-4: Doc chunks

Split the documents in chunks, according to their layout segmentation.

### 4.1 - Set Input/output Folder

In [7]:
STAGE = 2

input_folder = output_parquet_dir # previous output folder is the input folder for the current stage
output_folder =  output_chunk_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-2: Processing input='output/01_parquet_out' --> output='output/02_chunk_out'


### 4.2 -  Execute 

In [8]:
%%time 

# Import doc_json_chunk transform configuration
from doc_chunk_transform_ray import DocChunkRayTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc_chunk arguments
    # ...
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocChunkRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

15:46:51 INFO - doc_chunk parameters are : {'chunking_type': <chunking_types.DL_JSON: 'dl_json'>, 'content_column_name': 'contents', 'doc_id_column_name': 'document_id', 'dl_min_chunk_len': None, 'output_chunk_column_name': 'contents', 'output_source_doc_id_column_name': 'source_document_id', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox', 'chunk_size_tokens': 128, 'chunk_overlap_tokens': 30}
15:46:51 INFO - pipeline id pipeline_id
15:46:51 INFO - code location None
15:46:51 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
15:46:51 INFO - actor creation delay 0
15:46:51 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_chunk', 'job type': 'ray', 'job id': 'job_id'}
15:46:51 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_out output_folder - output/02_chunk_out
15:46:51 INFO - data factory data_ max_files -1, n_sample -1
15

✅ Stage:2 completed successfully
CPU times: user 1.24 s, sys: 480 ms, total: 1.72 s
Wall time: 35.5 s


### 4.3 - Inspect Generated output

We would see documents are split into many chunks

In [9]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Files processed : 2
Chunks created : 211
Input data dimensions (rows x columns)=  (2, 12)
Output data dimensions (rows x columns)=  (211, 16)


,filename,num_pages,num_tables,num_doc_elements,ext,hash,size,date_acquired,pdf_convert_time,source_filename,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id
128,attension.pdf,15,4,193,pdf,8b3ff3ebfc838c36fa10e6bbf90c1a5d2781f2d2583883...,135809,2024-10-08T15:45:38.327708,36.473885,attension.pdf,89c788c3-8f54-4809-8177-d90721f28587,Attention Is All You Need\nAidan N. Gomez ∗ † ...,$.main-text[8],1,"[234.82455444, 475.27728271, 339.99435425, 508...",3bdf2978bed016a6996cf37ffdd1fb96248cac96fccf70...
138,attension.pdf,15,4,193,pdf,8b3ff3ebfc838c36fa10e6bbf90c1a5d2781f2d2583883...,135809,2024-10-08T15:45:38.327708,36.473885,attension.pdf,89c788c3-8f54-4809-8177-d90721f28587,2 Background\nEnd-to-end memory networks are b...,$.main-text[25],2,"[107.37045288, 218.29309082, 505.6534729, 249....",aec7ea8282cbc4a7d6410e8d29df05025e75c084bf5098...
17,granite.pdf,28,17,348,pdf,4b4ee627dd6a21c1eb56205875a3970c3ad975864f3dcf...,655037,2024-10-08T15:46:23.368528,81.505457,granite.pdf,b5367177-e1c1-4643-9281-942f41bc5c87,"2 Data Collection\nIn this section, we describ...",$.main-text[39],4,"[107.18983459, 582.13323975, 504.2984314, 625....",c99319052c80981fe2c47fef9fc6110348b53f6faf3389...


## Step-5:  DOC ID generation

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set hash_column to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set int_id_column to the name of the column, where you want to store it. **This is a pre-requisite for fuzzy dedup** in the pipeline.

### 5.1 - Set Input/output Folder

In [10]:

STAGE  = 3

input_folder = output_chunk_dir # previous output folder is the input folder for the current stage
output_folder =  output_docid_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-3: Processing input='output/02_chunk_out' --> output='output/03_docid_out'


### 5.2 - Execute 

In [11]:
%%time 

from doc_id_transform_ray import DocIDRayTransformRuntimeConfiguration
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "chunk_hash",
    "doc_id_int_column": "chunk_id",
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(DocIDRayTransformRuntimeConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

15:47:11 INFO - Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'chunk_hash', 'int_column': 'chunk_id', 'start_id': 0}
15:47:11 INFO - pipeline id pipeline_id
15:47:11 INFO - code location None
15:47:11 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
15:47:11 INFO - actor creation delay 0
15:47:11 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_id', 'job type': 'ray', 'job id': 'job_id'}
15:47:11 INFO - data factory data_ is using local data access: input_folder - output/02_chunk_out output_folder - output/03_docid_out
15:47:11 INFO - data factory data_ max_files -1, n_sample -1
15:47:11 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
15:47:11 INFO - Running locally
2024-10-08 15:47:13,710	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(orchestrate pid=600

✅ Stage:3 completed successfully
CPU times: user 70.9 ms, sys: 118 ms, total: 189 ms
Wall time: 16 s


### 5.3 - Inspect Generated output

In [12]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (211, 16)
Output data dimensions (rows x columns)=  (211, 18)


,filename,num_pages,num_tables,num_doc_elements,ext,hash,size,date_acquired,pdf_convert_time,source_filename,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id,chunk_hash,chunk_id
198,attension.pdf,15,4,193,pdf,8b3ff3ebfc838c36fa10e6bbf90c1a5d2781f2d2583883...,135809,2024-10-08T15:45:38.327708,36.473885,attension.pdf,89c788c3-8f54-4809-8177-d90721f28587,6.3 English Constituency Parsing\nVinyals & Ka...,$.tables[3],10,"[144.17385864, 555.05541992, 467.28027344, 699...",2daf778770c8a3d86ef693fc90ab3505d1856ec4ad486e...,2daf778770c8a3d86ef693fc90ab3505d1856ec4ad486e...,198
74,granite.pdf,28,17,348,pdf,4b4ee627dd6a21c1eb56205875a3970c3ad975864f3dcf...,655037,2024-10-08T15:46:23.368528,81.505457,granite.pdf,b5367177-e1c1-4643-9281-942f41bc5c87,"6.1.5 RepoBench, CrossCodeEval: Repository-Lev...",$.main-text[163],13,"[107.2381134, 160.20976257, 505.74502563, 236....",9e47ead24fc356ae5aae90637990bcafb81064d2367e1a...,9e47ead24fc356ae5aae90637990bcafb81064d2367e1a...,74
41,granite.pdf,28,17,348,pdf,4b4ee627dd6a21c1eb56205875a3970c3ad975864f3dcf...,655037,2024-10-08T15:46:23.368528,81.505457,granite.pdf,b5367177-e1c1-4643-9281-942f41bc5c87,5 Instruction Tuning\nFinetuning code LLMs on ...,$.main-text[91],7,"[107.37421417, 126.16381836, 504.37268066, 214...",39f55cbc4100ba434b64586c17c9ab8d8841131c9214da...,39f55cbc4100ba434b64586c17c9ab8d8841131c9214da...,41


## Step-6: Exact Dedup

Remove documents having identical code to remove bias in the training data. On the content of each document, a SHA256 hash is computed,
followed by de-duplication of record having identical hashes.

### 6.1 -  Set Input/output Folder

In [13]:
STAGE  = 4

input_folder = output_docid_dir # previous output folder is the input folder for the current stage
output_folder =  output_exact_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-4: Processing input='output/03_docid_out' --> output='output/04_exact_dedupe_out'


### 6.2 - Execute 

In [14]:
%%time

# Import ededup transform configuration
from ededup_transform_ray import EdedupRayTransformRuntimeConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # ededup parameters
    "ededup_hash_cpu": 0.5,
    "ededup_num_hashes": 2,
    "ededup_doc_column": "contents",
    "ededup_doc_id_column": "chunk_hash",
    
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(EdedupRayTransformRuntimeConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

15:47:27 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'chunk_hash', 'use_snapshot': False, 'snapshot_directory': None, 'hash_cpu': 0.5, 'num_hashes': 2}
15:47:27 INFO - pipeline id pipeline_id
15:47:27 INFO - code location None
15:47:27 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
15:47:27 INFO - actor creation delay 0
15:47:27 INFO - job details {'job category': 'preprocessing', 'job name': 'ededup', 'job type': 'ray', 'job id': 'job_id'}
15:47:27 INFO - data factory data_ is using local data access: input_folder - output/03_docid_out output_folder - output/04_exact_dedupe_out
15:47:27 INFO - data factory data_ max_files -1, n_sample -1
15:47:27 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
15:47:27 INFO - Running locally
2024-10-08 15:47:29,753	INFO worker.py:1777 -- Started a local Ray instance. View the dash

✅ Stage:4 completed successfully
CPU times: user 67 ms, sys: 122 ms, total: 189 ms
Wall time: 16 s


### 6.3 - Inspect Generated output

In [15]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input chunks before exact dedupe : {input_df.shape[0]:,}")
print (f"Output chunks after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate chunks removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (211, 18)
Output data dimensions (rows x columns)=  (211, 19)
Input chunks before exact dedupe : 211
Output chunks after exact dedupe : 211
Duplicate chunks removed :   0


,filename,num_pages,num_tables,num_doc_elements,ext,hash,size,date_acquired,pdf_convert_time,source_filename,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id,chunk_hash,chunk_id,removed
23,granite.pdf,28,17,348,pdf,4b4ee627dd6a21c1eb56205875a3970c3ad975864f3dcf...,655037,2024-10-08T15:46:23.368528,81.505457,granite.pdf,b5367177-e1c1-4643-9281-942f41bc5c87,3 Model Architecture\nWe train a series of cod...,$.main-text[56],5,"[106.90156555, 463.37445068, 505.64907837, 507...",0aa927498f188fd1a97c481bee4a5581c197d44f37e5fa...,0aa927498f188fd1a97c481bee4a5581c197d44f37e5fa...,23,[]
208,attension.pdf,15,4,193,pdf,8b3ff3ebfc838c36fa10e6bbf90c1a5d2781f2d2583883...,135809,2024-10-08T15:45:38.327708,36.473885,attension.pdf,89c788c3-8f54-4809-8177-d90721f28587,Attention Visualizations Input-Input Layer5\nF...,$.main-text[181],13,"[107.35285187, 436.88726807, 504.46768188, 479...",0a545be8b488243477a72f2a991f3151d05af088a652a4...,0a545be8b488243477a72f2a991f3151d05af088a652a4...,208,[]
170,attension.pdf,15,4,193,pdf,8b3ff3ebfc838c36fa10e6bbf90c1a5d2781f2d2583883...,135809,2024-10-08T15:45:38.327708,36.473885,attension.pdf,89c788c3-8f54-4809-8177-d90721f28587,"4 Why Self-Attention\nAs noted in Table 1, a s...",$.main-text[83],6,"[107.27950287, 69.51050568, 504.59405518, 101....",f3e8a0d1cf72ab4028684f84672386fe4adbad4cdc5d0a...,f3e8a0d1cf72ab4028684f84672386fe4adbad4cdc5d0a...,170,[]


## Step-7: Fuzzy Dedup

Post exact deduplication, fuzzy deduplication is applied with
the goal of removing code files that may have slight variations and thereby unbiasing
the data further. Small variations are quite commonly seen in code data in the form
of variations in the values of variables, addittion of logging statements etc. Find near-
duplicate.

### 7.1 - Set Input/output Folder

In [16]:
## Input to this component is the output of doc_id generator component. 

STAGE  = 5

input_folder = output_exact_dedupe_dir # previous output folder is the input folder for the current stage
output_folder =  output_fuzzy_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-5: Processing input='output/04_exact_dedupe_out' --> output='output/05_fuzzy_dedupe_out'


### 7.2 - Execute 

In [17]:
%%time 

import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_ray import FdedupRayTransformConfiguration

# create parameters

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # Orchestration parameters
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "chunk_id",
    "fdedup_cluster_column": "chunk_hash",
    # infrastructure
    "fdedup_bucket_cpu": 0.3,
    "fdedup_doc_cpu": 0.3,
    "fdedup_mhash_cpu": 0.3,
    "fdedup_num_doc_actors": 1,
    "fdedup_num_bucket_actors": 1,
    "fdedup_num_minhash_actors": 1,
    "fdedup_num_preprocessors": 1,
    # fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.7, # between 0.0 to 1.0 ; smaller values tend to be more lenient in finding near dupes; close to 1.0 is more strict
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
}

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(FdedupRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

15:47:43 INFO - fuzzy dedup params are {'doc_column': 'contents', 'id_column': 'chunk_id', 'cluster_column': 'chunk_hash', 'bucket_cpu': 0.3, 'mhash_cpu': 0.3, 'doc_cpu': 0.3, 'num_doc_actors': 1, 'num_minhash_actors': 1, 'num_bucket_actors': 1, 'num_preprocessors': 1, 'num_permutations': 64, 'threshold': 0.7, 'shingles_size': 5, 'delimiters': ' ', 'snapshot_delay': 1, 'use_bucket_snapshot': False, 'use_doc_snapshot': False, 'random_delay_limit': 10, 'worker_options': {'num_cpus': 1}}
15:47:43 INFO - pipeline id pipeline_id
15:47:43 INFO - code location None
15:47:43 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
15:47:43 INFO - actor creation delay 0
15:47:43 INFO - job details {'job category': 'preprocessing', 'job name': 'fdedup', 'job type': 'ray', 'job id': 'job_id'}
15:47:43 INFO - data factory data_ is using local data access: input_folder - output/04_exact_dedupe_out output_folder - output/05_fuzzy_dedupe_out
15:47:43 INFO - data factory data_ max

✅ Stage:5 completed successfully
CPU times: user 106 ms, sys: 147 ms, total: 253 ms
Wall time: 30.6 s


### 7.3 - Inspect Generated output

In [18]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print ("Duplicate chunks removed  by fuzzy-dedupe:  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (211, 19)
Output data dimensions (rows x columns)=  (211, 19)
Duplicate chunks removed  by fuzzy-dedupe:   0


,filename,num_pages,num_tables,num_doc_elements,ext,hash,size,date_acquired,pdf_convert_time,source_filename,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id,chunk_id,removed,chunk_hash
105,granite.pdf,28,17,348,pdf,4b4ee627dd6a21c1eb56205875a3970c3ad975864f3dcf...,655037,2024-10-08T15:46:23.368528,81.505457,granite.pdf,b5367177-e1c1-4643-9281-942f41bc5c87,6.5 Math Reasoning\nTable 15 summarizes the re...,$.main-text[227],19,"[106.84754181, 127.60254669, 504.37875366, 171...",a5543da64e4d56508b18d26addd28131c63921a3a0f7a1...,105,[],-1
113,granite.pdf,28,17,348,pdf,4b4ee627dd6a21c1eb56205875a3970c3ad975864f3dcf...,655037,2024-10-08T15:46:23.368528,81.505457,granite.pdf,b5367177-e1c1-4643-9281-942f41bc5c87,"6.7 Model Robustness\nStarCoderBase-3B, Docstr...",$.tables[16],21,"[162.45884705, 471.86456299, 449.44735718, 668...",9d850c36997f8f8ebb4bed26cad319cd84c40a8c294660...,113,[],-1
20,granite.pdf,28,17,348,pdf,4b4ee627dd6a21c1eb56205875a3970c3ad975864f3dcf...,655037,2024-10-08T15:46:23.368528,81.505457,granite.pdf,b5367177-e1c1-4643-9281-942f41bc5c87,"2.3 HAP, PII, Malware Filtering\nTo reduce the...",$.main-text[45],4,"[107.02816772, 100.26654053, 505.24621582, 189...",3610c8f2db4587033885befe968d9ccbb40f7d5c97f21c...,20,[],-1


## Step-8:   Text encoding

Encode text for the vector storage.

### 8.1 - Set Input/output Folder

In [19]:
STAGE  = 6

input_folder = output_fuzzy_dedupe_dir
output_folder =  output_embeddings_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-6: Processing input='output/05_fuzzy_dedupe_out' --> output='output/06_embeddings_out'


### 8.2 - Execute

In [20]:
%%time 

from text_encoder_transform_ray import TextEncoderRayTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # text_encoder
    "text_encoder_model_name": MY_CONFIG.EMBEDDING_MODEL,
}

sys.argv = ParamsUtils.dict_to_req(d=params)
# create launcher
launcher = RayTransformLauncher(TextEncoderRayTransformConfiguration())
# Launch the ray actor(s) to process the input

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

15:48:15 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
15:48:15 INFO - pipeline id pipeline_id
15:48:15 INFO - code location None
15:48:15 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
15:48:15 INFO - actor creation delay 0
15:48:15 INFO - job details {'job category': 'preprocessing', 'job name': 'text_encoder', 'job type': 'ray', 'job id': 'job_id'}
15:48:15 INFO - data factory data_ is using local data access: input_folder - output/05_fuzzy_dedupe_out output_folder - output/06_embeddings_out
15:48:15 INFO - data factory data_ max_files -1, n_sample -1
15:48:15 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
15:48:15 INFO - Running locally
2024-10-08 15:48:17,403	INFO worker.py:1777 -- Started a local Ray instance.

✅ Stage:6 completed successfully
CPU times: user 341 ms, sys: 231 ms, total: 572 ms
Wall time: 27.7 s


### 8.3 - Inspect Generated output

In [21]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (211, 19)
Output data dimensions (rows x columns)=  (211, 20)


,filename,num_pages,num_tables,num_doc_elements,ext,hash,size,date_acquired,pdf_convert_time,source_filename,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id,chunk_id,removed,chunk_hash,embeddings
78,granite.pdf,28,17,348,pdf,4b4ee627dd6a21c1eb56205875a3970c3ad975864f3dcf...,655037,2024-10-08T15:46:23.368528,81.505457,granite.pdf,b5367177-e1c1-4643-9281-942f41bc5c87,6.2 Code Explanation and Fixing\nWhile most of...,$.main-text[173],14,"[107.09112549, 340.27554321, 505.74041748, 427...",2c34041020831bf879b275591bf69d51d3a3935ac9eae1...,78,[],-1,"[-0.013920594, -0.021673767, 0.06490408, -0.03..."
99,granite.pdf,28,17,348,pdf,4b4ee627dd6a21c1eb56205875a3970c3ad975864f3dcf...,655037,2024-10-08T15:46:23.368528,81.505457,granite.pdf,b5367177-e1c1-4643-9281-942f41bc5c87,"6.4 Code Reasoning, Understanding and Executio...",$.main-text[209],18,"[106.99939728, 242.3865509, 505.73922729, 373....",c1c7b810f73360048f595898c49a356490fe9708659863...,99,[],-1,"[-0.05579955, -0.078981526, -0.016305814, 0.05..."
86,granite.pdf,28,17,348,pdf,4b4ee627dd6a21c1eb56205875a3970c3ad975864f3dcf...,655037,2024-10-08T15:46:23.368528,81.505457,granite.pdf,b5367177-e1c1-4643-9281-942f41bc5c87,6.3 Code Editing and Translation\nTable 11: Pa...,$.main-text[188],16,"[196.31352234, 697.56256104, 414.67791748, 708...",ab5dee480c14bc13a764485e07b33035e6e052d138c231...,86,[],-1,"[0.029248616, -0.007842009, -0.0911203, -0.037..."


## Step-9: Copy output to final output dir

In [22]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_folder, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_folder}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output/06_embeddings_out' --> 'output/output_final'
